In [1]:
import numpy as np
import pandas as pd
import math
import random
import matplotlib.pyplot as plt
from numba import jit, prange
import networkx as nx
from matplotlib.lines import Line2D
from sklearn.linear_model import LinearRegression
from scipy import optimize, stats
from pylab import *
from scipy.ndimage import measurements

In [2]:
@jit(nopython = True)
def init_lattice(L, perc):
    '''
    Returns an initial configuration of size N with N*perc initial infected chosen randomly
    
    INPUTs:
    -size of the lattice
    -percentage of initial infected
    OUTPUTs:
    -lattice
    '''
    N = L**2
    lattice = np.full(N, 0)
    initial_infected = int(N*perc)
    for i in range(initial_infected): lattice[i] = 1
    np.random.shuffle(lattice)
    return lattice

In [6]:
def plot_lattice(lattice, n_step, nodesize = 100):
    '''
    Plot a configuration
    '''
    fig = plt.figure(figsize = (7, 7))
    N = len(lattice)
    L = int(np.sqrt(N))
    #L = lattice.shape[0]
    G = nx.grid_2d_graph(L, L)
    pos = dict( (n, n) for n in G.nodes() )
    colors = np.full(N, ["blue"])
    #print(pos)
    for i in range(N):
        if lattice[i] == 1: colors[i] = "red"
        elif lattice[i] == 2: colors[i] = "grey"
    
    colors = (colors.reshape(L, L).T).reshape(-1, N)[0]
    legend_elements = [Line2D([0], [0], marker = "o", color="w",label="Scatter",
                                 markerfacecolor = "blue", markersize = 14),
                           Line2D([0], [0], marker="o", color="w", label="Scatter",
                                  markerfacecolor= "red", markersize=14),
                      Line2D([0], [0], marker="o", color="w", label="Scatter",
                                  markerfacecolor= "gray", markersize=14)]
    
    nx.draw_networkx(G, pos = pos, with_labels = False, labels = None,
                         node_color = colors, node_size = nodesize)
    plt.title("t = "+str(n_step)+", c = 0.17")#, weight = "bold", size = 16)
    plt.axis('off')
    plt.gca().invert_yaxis()
    #plt.gca().invert_xaxis()
    #plt.legend(legend_elements, ["Healthy", "Infected", "Recovered"], bbox_to_anchor=(1, 1), loc="upper left", fontsize = 14)
    #plt.show()
    plt.close()
    fig.savefig("presentazione/GIF_SIR/sir-"+str(n_step)+".pdf", transparent = True, bbox_inches = 'tight',
    pad_inches = 0)
    return

In [7]:
@jit(nopython = True)
def build_I(lattice):
    N = len(lattice)
    I = np.array([2*N], dtype = np.int64)
    for n in prange(N):
        if lattice[n] == 1: I = np.append(I, n)
    return I


@jit(nopython = True)
def SIR(L, I, R, rec_rate):
    
    N = L**2
    c = rec_rate#/(rec_rate + inf_rate)
    n_infected = len(I)-1
    #S_new = S.copy()
    #I_new = I.copy()
    #R_new = I.copy()
    
    if (len(I)-1) > 0:
        for s in prange(n_infected):
        #for s in range(1, len(I)):
            i = np.random.randint(1, len(I))
            #i = s
            random_site = I[i]
            
            if np.random.uniform(0., 1.) < c:
                #idx = I.index(random_site)
                I[i] = I[-1]
                I = I[:-1]
                R = np.append(R, random_site)
                #n_infected -= 1
            else: 
                nbrs = [(random_site-1)%N, (random_site+1)%N, (random_site+L)%N, (random_site-L)%N]
                j = np.random.randint(0, 4)
                random_nbr = nbrs[j]
                if np.any(R == random_nbr)==False and np.any(I == random_nbr)==False:
                    #idx = np.where(S == random_nbr)[0]
                    #S[idx] = S[-1]
                    #S = S[:-1]
                    I = np.append(I, random_nbr)
                    #n_infected+=1
            #print(I)
            #print(R)
            #print("\n")
    return I, R

In [8]:
L = 30
N = L**2
lattice = init_lattice(L, 1/N)
I = build_I(lattice)
I1 = I[1:]
R = np.array([2*N], dtype = np.int64)
new_lattice = np.zeros(N)
for i in I1:
    new_lattice[i] = 1
    #print(new_lattice.reshape(30,30))
plot_lattice(new_lattice, n_step=0, nodesize = 50)

n = 1
while (len(I)-1)>0:
    I, R = SIR(L, I, R, 0.17)
    I1 = I[1:]
    R1 = R[1:]
    new_lattice = np.zeros(N)
    for i in I1:
        new_lattice[i] = 1
    for r in R1:
        new_lattice[r] = 2
    plot_lattice(new_lattice, n, 50)
    n+=1

In [28]:
import os
import imageio

png_dir = 'presentazione/GIF_SIR/'
images = []

names = []

# Get names
for file_name in sorted(os.listdir(png_dir)):
    if file_name.endswith('.png'):
        names.append(file_name)
        
#Sort names
names = sorted(names, key=lambda x: int(x.split('.')[0]))

#Create gif
for name in names:
    file_path = os.path.join(png_dir, name)
    images.append(imageio.imread(file_path))
imageio.mimsave('movie.gif', images, fps = 2.5)